In [360]:
!pip install pdfminer pypdf2 tqdm openpyxl pandas

Defaulting to user installation because normal site-packages is not writeable


In [184]:
!exa -l data

drwxr--r-- - marcin 24 Jul  1:51 Alabama
drwxr--r-- - marcin 27 Jul 17:57 Arkansas
drwxr--r-- - marcin  4 May 23:02 Berkeley
drwxr--r-- - marcin  3 Dec 17:20 Bowdoin
drwxr--r-- - marcin  3 Dec 15:32 Clemson
drwxr--r-- - marcin  3 Dec 17:09 Colby
drwxr--r-- - marcin 27 May 18:07 Colorado - Boulder
drwxr--r-- - marcin 29 Jul 18:20 Colorado State
drwxr--r-- - marcin  3 Dec 16:11 CSU Fresno
drwxr--r-- - marcin  3 Dec 18:00 Duke
drwxr--r-- - marcin 13 Jul 19:05 George Mason
drwxr--r-- - marcin  3 Dec 17:43 Gonzaga
drwxr--r-- - marcin  3 Dec 17:25 Maine - Augusta
drwxr--r-- - marcin  3 Dec 17:47 Marshall
drwxr--r-- - marcin 27 May 18:22 Ohio State
drwxr--r-- - marcin 29 Jul 19:35 Old Dominion
drwxr--r-- - marcin  3 Dec 16:31 Oregon
drwxr--r-- - marcin  3 Dec 20:01 Swarthmore
drwxr--r-- - marcin  3 Dec 17:01 UCLA
drwxr--r-- - marcin 27 May 18:23 West Virginia
drwxr--r-- - marcin  3 Dec 20:02 William Carey


In [149]:
# !exa data -T

In [53]:
import io
import re
import os
import sys
from collections import defaultdict

import pandas as pd
from tqdm.notebook import trange, tqdm

def find(phrase, text, backward=10, forward=1000):
    """ 
    finds phrase in text, and returns surrounding 
    characters (number of characters ranging from
    backward to forward)
    """
    idx = text.index(phrase)
    return text[idx-backward:idx+forward]

In [138]:
# from pprint import pprint
# import PyPDF2
# def walk(obj, fnt, emb):
#     '''
#     If there is a key called 'BaseFont', that is a font that is used in the document.
#     If there is a key called 'FontName' and another key in the same dictionary object
#     that is called 'FontFilex' (where x is null, 2, or 3), then that fontname is 
#     embedded.
    
#     We create and add to two sets, fnt = fonts used and emb = fonts embedded.
#     '''
#     if not hasattr(obj, 'keys'):
#         return None, None
#     fontkeys = set(['/FontFile', '/FontFile2', '/FontFile3'])
#     if '/BaseFont' in obj:
#         fnt.add(obj['/BaseFont'])
#     if '/FontName' in obj:
#         if [x for x in fontkeys if x in obj]:# test to see if there is FontFile
#             emb.add(obj['/FontName'])

#     for k in obj.keys():
#         walk(obj[k], fnt, emb)

#     return fnt, emb# return the sets for each page


# fname = 'data/Ohio State/course_catalog_2010_2011.pdf'
# pdf = PdfFileReader(fname)
# fonts = set()
# embedded = set()
# for page in pdf.pages:
#     obj = page.getObject()
#     # updated via this answer:
#     # https://stackoverflow.com/questions/60876103/use-pypdf2-to-detect-non-embedded-fonts-in-pdf-file-generated-by-google-docs/60895334#60895334 
#     # in order to handle lists inside objects. Thanks misingnoglic !
#     # untested code since I don't have such a PDF to play with.
#     if type(obj) == PyPDF2.generic.ArrayObject:  # You can also do ducktyping here
#         for i in obj:
#             if hasattr(i, 'keys'):
#                 f, e = walk(i, fonts, embedded_fonts)
#                 fonts = fonts.union(f)
#                 embedded = embedded.union(e)
#     else:
#         f, e = walk(obj['/Resources'], fonts, embedded)
#         fonts = fonts.union(f)
#         embedded = embedded.union(e)

# unembedded = fonts - embedded
# print ('Font List')
# pprint(sorted(list(fonts)))
# if unembedded:
#     print( '\nUnembedded Fonts')
#     pprint(unembedded)

In [150]:
from PyPDF2 import PdfFileReader

def pdf2txt(path, pages=None):
    txt = ''
    pdf = PdfFileReader(str(path))
    for page in pdf.pages:
        txt += page.extractText()
        if pages != None: pages -= 1
        if pages == 0: break
    return txt

In [105]:
from pdfminer.pdfinterp import (PDFResourceManager, PDFPageInterpreter)
from pdfminer.layout import LAParams, LTTextBox,LTChar, LTFigure
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfdocument import PDFDocument


def pdf_to_text(path, pages=None):
    with open(path, 'rb') as fp:
        rsrcmgr = PDFResourceManager()
        outfp = io.StringIO()
        laparams = LAParams()
        device = TextConverter(rsrcmgr, outfp, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.get_pages(fp):
            interpreter.process_page(page)
            if pages != None:
                pages -= 1
                if pages == 0: break
    text = outfp.getvalue()
    return text

class PdfMinerWrapper(object):
    """
    Usage:
    with PdfMinerWrapper('2009t.pdf') as doc:
        for page in doc:
           #do something with the page
    """
    def __init__(self, pdf_doc, pdf_pwd=""):
        self.pdf_doc = pdf_doc
        self.pdf_pwd = pdf_pwd
    def __enter__(self):
        #open the pdf file
        self.fp = open(self.pdf_doc, 'rb')
        # create a parser object associated with the file object
        parser = PDFParser(self.fp)
        # create a PDFDocument object that stores the document structure
        doc = PDFDocument(parser)
        # connect the parser and document objects
        parser.set_document(doc)
        self.doc=doc
        return self
    
    def _parse_pages(self):
        rsrcmgr = PDFResourceManager()
        laparams = LAParams(char_margin=3.5, all_texts = True)
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
    
        for page in PDFPage.create_pages(self.doc):
            interpreter.process_page(page)
            # receive the LTPage object for this page
            layout = device.get_result()
            # layout is an LTPage object which may contain child objects like LTTextBox, LTFigure, LTImage, etc.
            yield layout
    def __iter__(self): 
        return iter(self._parse_pages())
    
    def __exit__(self, _type, value, traceback):
        self.fp.close()
            
def get_departments(path, pages=27 department_font_size=17):
    department_courses = defaultdict(str)
    department_name = ''
    past_department_name = False
    with PdfMinerWrapper(path) as doc:
        for page in doc:     
            if pages != None and page.pageid > pages: break
            for tbox in page:
                if not isinstance(tbox, LTTextBox): continue
                for obj in tbox:
                    for c in obj:
                        if not isinstance(c, LTChar): continue
                        if c.size > department_font_size:
                            if past_department_name: department_name = ''
                            department_name += c.get_text()
                            past_department_name = False
                        else:
                            past_department_name = True
                    department_courses[department_name] += obj.get_text()

    return department_courses


In [33]:
done = False
for directory,_,filenames in os.walk('data'):
    if directory.lstrip('data/') in parse_only:
        for filename in filenames:
            data = pdf_to_text(directory+'/'+filename)
            done = True
            break
    if done: break

# Arkansas

In [152]:
arkansas_sample = pdf_to_text('data/Arkansas/2019-2020.pdf')

In [153]:
pd.DataFrame(re.findall(r"(\n\n?[A-Z]{4}\s[0-9]{3,4}\.[\(\w\s.\n,\.:-]*)",arkansas_sample))

,0
0,\nENGL 0002.\n\n
1,\nMATH 1203.\n\nStudents will be required to r...
2,\nCOMM 1313. Both the written and oral (a five...
3,\nPLSC 2003.\n\n98 University Core\n\n...
4,\nANTH 1023. At least 9 hours must be in cours...
...,...
5026,\n\nWLIT 5623. The Bible as Literature. 3 Hour...
5027,\n\nWLIT 5993. African Literature. 3 Hours.\n(...
5028,\n\nWLIT 6703. Psychoanalysis and Culture. 3 H...
5029,"\n\nWLIT 6713. Literature of Spain, 711-1615 C..."


# Bowdowin

In [110]:
# pdf2txt doesnt work
bowdowin_sample = pdf_to_text("data/Bowdoin/Bowdoin_2019-2020.pdf")
bs = bowdowin_sample

In [346]:
# find = "AFRS 1012"
# bs[bs.index(find)-10:bs.index(find)+2000]
bowdoin_dfs = []
for file in tqdm(os.listdir('data/Bowdoin')):
    text = pdf_to_text(f"data/Bowdoin/{file}")
    if '202' in file:
        data = [{'id':r[1],'title':r[3],'sub':r[4],'desc':r[6],'num':r[5],'catalog':file} 
            for r in re.findall(
                r"("
                    r"\n\n([A-Z]{4}(\s|\\xa0)[0-9]{4})"  # course id
                    r"(.*?)\n"                           # course title
                    r"(.*?)"                             # course subtitle
                    r"\s([0-9]{1,3})\."                  # course enrolment limit
                    r"((?:(?![\n]{1}[A-Z]{4}).)*)"       # course description 
                                                          # UNTIL next course id 
                                                          # but not including it
                r")",
                text,
                re.DOTALL)
        ]
    else: #pre 2020
        data = [{'id':r[1],'title':r[3],'sub':None,'desc':r[4],'num':None,'catalog':file} 
                for r in re.findall(
                r"("
                    r"\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)"  # course id
                    r"(.*?)\n"                           # course title
                    r"((?:(?!\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)).)*)"  # UNTIL next course id but not including
                r")",
                text,
                re.DOTALL)
        ]
        
    bowdoin_dfs.append(pd.DataFrame(data))

In [365]:
df_bowdoin

,id,title,sub,desc,num,catalog
0,10b,Racism. Fall 2010. Spring 2011. Roy Partridge....,None,12c. Affirmative Action and United States Soci...,None,Bowdoin_2010-2011.pdf
1,13c,From Montezuma to Bin Laden: Globalization and...,None,Gordon. (Same as History 16.),None,Bowdoin_2010-2011.pdf
2,16c,Fictions of Freedom. Fall 2010. Tess Chakkalak...,None,[25c. The Civil War in Film. (Same as History ...,None,Bowdoin_2010-2011.pdf
3,101c,Introduction to Africana Studies. Every fall. ...,None,Focuses on major humanities and social science...,None,Bowdoin_2010-2011.pdf
4,107c,ESD. Introduction to the Black Novel in the U...,None,Chakkalakal.Introduces students to the literar...,None,Bowdoin_2010-2011.pdf
...,...,...,...,...,...,...
840,320c,Theater Styles. Every third year. Fall 2009. D...,None,An advanced acting class that explores issues ...,None,Bowdoin_2009-2010.pdf
841,321c,Comedy in Performance. Every third year. Fall ...,None,"Looks at several facets of comedy on stage, fr...",None,Bowdoin_2009-2010.pdf
842,322c,Ensemble: Theater and Dance Collaborative...,None,DePaRtMent.Experienced theater and dance stude...,None,Bowdoin_2009-2010.pdf
843,323c,Acting Shakespeare. Spring 2010. abiGail killeen.,None,An advanced-level acting course dedicated to t...,None,Bowdoin_2009-2010.pdf


In [368]:
df_bowdoin = pd.concat(bowdoin_dfs)
df_bowdoin['title'] = df_bowdoin['title'].str.replace('\n','')
# df_bowdoin['title'] = df_bowdoin['title'].str.replace(r'\x','')
df_bowdoin['desc'] = df_bowdoin['desc'].str.replace('\n','')
# df_bowdoin['desc'] = df_bowdoin['desc'].str.replace(r'\x','')
df_bowdoin = df_bowdoin.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)
with pd.ExcelWriter('data/bowdoin.xlsx') as writer: 
    df_bowdoin.to_excel(writer, sheet_name='data')
df_bowdoin

,id,title,sub,desc,num,catalog
0,10b,Racism. Fall 2010. Spring 2011. Roy Partridge....,None,12c. Affirmative Action and United States Soci...,None,Bowdoin_2010-2011.pdf
1,13c,From Montezuma to Bin Laden: Globalization and...,None,Gordon. (Same as History 16.),None,Bowdoin_2010-2011.pdf
2,16c,Fictions of Freedom. Fall 2010. Tess Chakkalak...,None,[25c. The Civil War in Film. (Same as History ...,None,Bowdoin_2010-2011.pdf
3,101c,Introduction to Africana Studies. Every fall. ...,None,Focuses on major humanities and social science...,None,Bowdoin_2010-2011.pdf
4,107c,ESD. Introduction to the Black Novel in the U...,None,Chakkalakal.Introduces students to the literar...,None,Bowdoin_2010-2011.pdf
...,...,...,...,...,...,...
840,320c,Theater Styles. Every third year. Fall 2009. D...,None,An advanced acting class that explores issues ...,None,Bowdoin_2009-2010.pdf
841,321c,Comedy in Performance. Every third year. Fall ...,None,"Looks at several facets of comedy on stage, fr...",None,Bowdoin_2009-2010.pdf
842,322c,Ensemble: Theater and Dance Collaborative...,None,DePaRtMent.Experienced theater and dance stude...,None,Bowdoin_2009-2010.pdf
843,323c,Acting Shakespeare. Spring 2010. abiGail killeen.,None,An advanced-level acting course dedicated to t...,None,Bowdoin_2009-2010.pdf


# Ohio State

In [139]:
ohio_sample = pdf2txt("data/Ohio State/course_catalog_2010_2011.pdf")

In [140]:
find("Accounting and Management", ohio_sample)

'aded S/U.\nAccounting and Management \nInformation Systems\n 211Introduction to AccountingUGRD5\nPreparation and use of accounting reports for business entities; focus on uses of \naccounting for external reporting, emphasizing accounting as a provider of financial \ninformation.\nSu, Au, Wi, Sp Qtrs. 2 2-hr cl. Prereq or concur: Econ 200 or equiv. Not open to \n\nstudents with credit for 201 or 310 or equiv. This course is available for EM credit.\n 211HHonors Introduction to Accounting IUGRD5\nStructure of accounting information; role of accounting and auditing in markets and \n\norganizations; preparation and use of accounti\nng reports. Uses distinctive instructor-\nauthored materials and writing assignments, and emphasizes oral class participation.\n\nAu Qtr. 2 2-hr cl. Prereq: Permission of Honors Accounting Director.\n 212Introduction to AccountingUGRD5\nThe uses of accounting reports in management decisions and in control of business \n\noperations; focus is on the internal us

In [154]:
ohio_dfs = []
for file in tqdm(os.listdir('data/Ohio State')):
    # better to use pdf2txt, otherwise the number to the left of letter U
    # sometimes flotes away from the course it is assigned to
    ohs2 = pdf2txt(f"data/Ohio State/{file}")

    data = [{'id':r[1],'title':r[6],'desc':r[7],'num':r[8],'catalog':file} 
     for r 
     in re.findall(
         r"("
            r"\n("                           # course ID
                r"([0-9]{4}\.[0-9A-Z]{3})"     # e.g. 2367.07S
                r"|([0-9]{4}\.[0-9]{2})"       # e.g. 2367.04
                r"|([0-9]{4}[H])"              # e.g. 4780H
                r"|([0-9]{4})"                 # e.g. 3080
            r")"
           r"\n([A-Za-z\s]*)"                # course title
           r"\nU\n"                          # letter U
           r"(.*?)"                          # course desrciption
           r"\n([0-9]{1,2})"                 # number to the right of letter U
         r")",
         ohs2, 
         re.DOTALL) 
    ]
    ohio_dfs.append(pd.DataFrame(data))    

In [155]:
df_ohio = pd.concat(ohio_dfs)
df_ohio['title'] = df_ohio['title'].str.replace('\n','')
df_ohio['desc'] = df_ohio['desc'].str.replace('\n','')
with pd.ExcelWriter('data/ohio.xlsx') as writer: 
    df_ohio.to_excel(writer, sheet_name='data')
df_ohio

,id,title,desc,num,catalog
0,2000,Foundations of Accounting,A survey of accounting for non-business studen...,3,course_catalog_2015_2016.pdf
1,2200,Introduction to Accounting I,Preparation and use of accounting reports for ...,3,course_catalog_2015_2016.pdf
2,2200H,Introduction to Accounting I,Structure and use of financial reports; role o...,3,course_catalog_2015_2016.pdf
3,2300,Introduction to Accounting II,The uses of accounting reports by managers to ...,20,course_catalog_2015_2016.pdf
4,2300H,Introduction to Accounting II,Use of accounting information in management co...,3,course_catalog_2015_2016.pdf
...,...,...,...,...,...
3899,4721,Studies in Yiddish Literature,"Advanced study of specific literary periods, f...",3,Course Bulletin 2019-2020.pdf
3900,4998,Undergraduate Research,Investigation of problems in various fields of...,1,Course Bulletin 2019-2020.pdf
3901,4998H,Honors Research,Investigation of problems in various fields of...,3,Course Bulletin 2019-2020.pdf
3902,4999,Undergraduate Research Thesis,Research for undergraduate thesis.Prereq: Perm...,1,Course Bulletin 2019-2020.pdf


### Departments

In [156]:
departments_mappings = []
for file in tqdm(os.listdir('data/Ohio State')):
    mapping = get_departments(f"data/Ohio State/{file}")
    cleaned_mapping = dict()
    department_to_courses_ids_mapping = dict()
    
    for k,v in mapping.items():
        if len(v) > 50:
            cleaned_mapping[k] = v
            
    
    for k,v in cleaned_mapping.items():
        for r in re.findall(r'\n [0-9\.H]{2,7}\n', v):
            department_to_courses_ids_mapping[r.strip()] = k
            
    departments_mappings.append(pd.Series(department_to_courses_ids_mapping))

IndexError: list index out of range

In [157]:
department_to_courses_ids_mapping

{}

In [158]:
file

'course_catalog_2014_2015.pdf'